In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('./xclip'))
sys.path.insert(0, module_path)

In [2]:
import cv2
import argparse
import torch
import torchvision.transforms as T
from xclip.models.repnet import RepNet
from xclip.models.cct import CrossFrameCommunicationTransformer
from xclip.models.cct import load
from xclip.utils.logger import create_logger
import numpy as np


In [3]:
STRIDES = [1,2,3]
VIDEO_PATH = "/Users/kanantharaman/Downloads/detectron/Data/videos/test/push-up/push-up_5.mp4"

In [4]:
transform = T.Compose([
        T.ToPILImage(),
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=0.5, std=0.5),
    ])
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
raw_frames, frames = [], []
while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame is None:
            break
        raw_frames.append(frame)
        frame = transform(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        frames.append(frame)
cap.release()

In [5]:
model = RepNet()
state_dict = torch.load('./xclip/repnet_weights.pth', map_location='cpu')
model.load_state_dict(state_dict, strict=False)
model.eval()
model.to('cpu')

RepNet(
  (tsm_conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (period_length_head): Sequential(
    (0): TranformerLayer(
      (input_projection): Linear(in_features=2048, out_features=512, bias=True)
      (transformer_layer): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
    (1): Linear(in_features=512, out_features=512, bias=True)
    

In [6]:
state_dict1 = torch.load('./xclip/best.pth', map_location='cpu')


In [24]:
state_dict1['model'].keys()

odict_keys(['positional_embedding', 'text_projection', 'logit_scale', 'prompts_visual_proj', 'prompts_generator.alpha', 'prompts_generator.norm.weight', 'prompts_generator.norm.bias', 'prompts_generator.decoder.0.cross_attn.q_proj.weight', 'prompts_generator.decoder.0.cross_attn.k_proj.weight', 'prompts_generator.decoder.0.cross_attn.v_proj.weight', 'prompts_generator.decoder.0.cross_attn.proj.weight', 'prompts_generator.decoder.0.cross_attn.proj.bias', 'prompts_generator.decoder.0.norm1.weight', 'prompts_generator.decoder.0.norm1.bias', 'prompts_generator.decoder.0.norm3.weight', 'prompts_generator.decoder.0.norm3.bias', 'prompts_generator.decoder.0.mlp.0.weight', 'prompts_generator.decoder.0.mlp.0.bias', 'prompts_generator.decoder.0.mlp.3.weight', 'prompts_generator.decoder.0.mlp.3.bias', 'prompts_generator.decoder.1.cross_attn.q_proj.weight', 'prompts_generator.decoder.1.cross_attn.k_proj.weight', 'prompts_generator.decoder.1.cross_attn.v_proj.weight', 'prompts_generator.decoder.1.c

In [7]:
stride_frames = frames[::1]
stride_frames = stride_frames[:(len(stride_frames) // 64) * 64]
logger = create_logger('./')

embed_model, _ = load(None, 'ViT-B/16', 
                         device="cpu", jit=False, 
                         T=8, logger=logger
                        )

[2024-10-31 14:48:25 root](cct.py 168): INFO load pretrained CLIP: _IncompatibleKeys(missing_keys=['class_embedding', 'positional_embedding', 'proj', 'conv1.weight', 'ln_pre.weight', 'ln_pre.bias', 'transformer.resblocks.0.message_fc.weight', 'transformer.resblocks.0.message_fc.bias', 'transformer.resblocks.0.message_ln.weight', 'transformer.resblocks.0.message_ln.bias', 'transformer.resblocks.0.message_attn.in_proj_weight', 'transformer.resblocks.0.message_attn.in_proj_bias', 'transformer.resblocks.0.message_attn.out_proj.weight', 'transformer.resblocks.0.message_attn.out_proj.bias', 'transformer.resblocks.0.attn.in_proj_weight', 'transformer.resblocks.0.attn.in_proj_bias', 'transformer.resblocks.0.attn.out_proj.weight', 'transformer.resblocks.0.attn.out_proj.bias', 'transformer.resblocks.0.ln_1.weight', 'transformer.resblocks.0.ln_1.bias', 'transformer.resblocks.0.mlp.c_fc.weight', 'transformer.resblocks.0.mlp.c_fc.bias', 'transformer.resblocks.0.mlp.c_proj.weight', 'transformer.resb

In [24]:

stride_frames = frames[::1]
stride_frames = stride_frames[:(len(stride_frames) // 64) * 64]
print(len(stride_frames))
stride_frames = torch.stack(stride_frames, axis=0).unflatten(0, (-1, 64))
print(stride_frames.shape)
b,t,c,h,w = stride_frames.size()
image = stride_frames.reshape(-1,c,h,w)
image.shape
#with torch.no_grad():
#    x=embed_model(image)

128
torch.Size([2, 64, 3, 224, 224])


torch.Size([128, 3, 224, 224])

In [118]:
for stride in STRIDES:
        # Apply stride
        stride_frames = x[0][::stride]
        stride_frames = stride_frames[:(len(stride_frames) // 64) * 64]
        if len(stride_frames) < 64:
            continue # Skip this stride if there are not enough frames
        stride_frames = stride_frames.unflatten(0, (-1, 64))#torch.stack(stride_frames, axis=0)#.unflatten(0, (-1, 64))#.movedim(1, 2) # Convert to N x C x D x H x W
        #stride_frames = stride_frames.to('cpu')
        print(stride_frames.shape)
        dist = torch.cdist(stride_frames[0], stride_frames[0], p=2)**2

        

torch.Size([2, 64, 512])
torch.Size([1, 64, 512])


In [139]:
cd =dist.numpy()
np.fill_diagonal(cd, np.nan)

In [119]:
def plot_heatmap(dist: np.ndarray, log_scale: bool = False) -> np.ndarray:
    """Plot the temporal self-similarity matrix into an OpenCV image."""
    np.fill_diagonal(dist, np.nan)
    if log_scale:
        dist = np.log(1 + dist)
    dist = -dist # Invert the distance
    zmin, zmax = np.nanmin(dist), np.nanmax(dist)
    heatmap = (dist - zmin) / (zmax - zmin) # Normalize into [0, 1]
    heatmap = np.nan_to_num(heatmap, nan=1)
    heatmap = np.clip(heatmap * 255, 0, 255).astype(np.uint8)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_VIRIDIS)
    return heatmap

In [120]:
tsm_img = plot_heatmap(dist.numpy(), log_scale=True)

/var/folders/fn/gqsn1b5x75x3nstgrhp06xtw0000gp/T/ipykernel_55198/3725310736.py:5: RuntimeWarning: invalid value encountered in log
  dist = np.log(1 + dist)


In [127]:
cv2.imwrite('./tsm.png',tsm_img)

True

In [168]:
np.fill_diagonal(cd, np.nan)
dist = -dist # Invert the distance
zmin, zmax = np.nanmin(dist), np.nanmax(dist)
heatmap = (dist - zmin) / (zmax - zmin) # Normalize into [0, 1]
heatmap = np.nan_to_num(heatmap, nan=1)
#heatmap = np.clip(heatmap * 255, 0, 255).astype(np.uint8)
#heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_VIRIDIS)

In [170]:
heatmap[0]

array([1.        , 0.09033645, 0.10705061, 0.12384808, 0.12519787,
       0.13238014, 0.11348297, 0.09645219, 0.09696878, 0.18128946,
       0.13767934, 0.2334483 , 0.2649769 , 0.41192153, 0.47224575,
       0.584379  , 0.6739655 , 0.72204167, 0.78493214, 0.7981803 ,
       0.7918645 , 0.77593356, 0.76390195, 0.66958284, 0.5973604 ,
       0.46881294, 0.38729185, 0.3783099 , 0.20096985, 0.1535103 ,
       0.06732324, 0.12869735, 0.16372544, 0.17607358, 0.19958673,
       0.20331949, 0.18228932, 0.15489343, 0.08365411, 0.1345298 ,
       0.19450416, 0.24129711, 0.26795977, 0.4320185 , 0.5005415 ,
       0.5671149 , 0.6029262 , 0.6263227 , 0.7232248 , 0.7322567 ,
       0.7386558 , 0.7334233 , 0.73715603, 0.74088883, 0.7367728 ,
       0.7395057 , 0.72092515, 0.70969355, 0.6143078 , 0.635238  ,
       0.5704644 , 0.51817226, 0.37066105, 0.3526804 ], dtype=float32)

In [9]:
best_stride, best_confidence, best_period_length, best_period_count, best_periodicity_score, best_embeddings = None, None, None, None, None, None
for stride in STRIDES:
        # Apply stride
        stride_frames = x[0][::stride]
        stride_frames = stride_frames[:(len(stride_frames) // 64) * 64]
        if len(stride_frames) < 64:
            continue # Skip this stride if there are not enough frames
        stride_frames = stride_frames.unflatten(0, (-1, 64))
        #stride_frames = torch.stack(stride_frames, axis=0)#.unflatten(0, (-1, 64))#.movedim(1, 2) # Convert to N x C x D x H x W
        #stride_frames = stride_frames.to('cpu')
        #print(stride_frames.shape)
        raw_period_length, raw_periodicity_score, embeddings = [], [], []
        with torch.no_grad():
            for i in range(stride_frames.shape[0]):  # Process each batch separately to avoid OOM
                batch_period_length, batch_periodicity, batch_embeddings = model(stride_frames)
                raw_period_length.append(batch_period_length[0].cpu())
                raw_periodicity_score.append(batch_periodicity[0].cpu())
                embeddings.append(batch_embeddings[0].cpu()) 
        # Post-process results
        raw_period_length, raw_periodicity_score, embeddings = torch.cat(raw_period_length), torch.cat(raw_periodicity_score), torch.cat(embeddings)
        confidence, period_length, period_count, periodicity_score = model.get_counts(raw_period_length, raw_periodicity_score, stride)
        if best_confidence is None or confidence > best_confidence:
            best_stride, best_confidence, best_period_length, best_period_count, best_periodicity_score, best_embeddings = stride, confidence, period_length, period_count, periodicity_score, embeddings
        if best_stride is None:
            raise RuntimeError('The stride values used are too large and nove 64 video chunk could be sampled. Try different values for --strides.')
        print(f'Predicted a period length of {best_period_length/fps:.1f} seconds (~{int(best_period_length)} frames) with a confidence of {best_confidence:.2f} using a stride of {best_stride} frames.')



Predicted a period length of 33366.7 seconds (~1000000 frames) with a confidence of 0.00 using a stride of 1 frames.
Predicted a period length of 33366.7 seconds (~1000000 frames) with a confidence of 0.00 using a stride of 1 frames.


In [15]:
batch_period_length[0][0]

tensor([-9.6993,  3.0649, -0.2384,  0.0580, -1.2835,  0.5521,  0.5216,  1.0147,
        -0.8328,  0.8525,  2.1904,  0.4364, -1.1787, -0.1614, -0.7137,  1.3902,
        -1.3939, -0.1937, -2.3317, -0.1535, -1.8956, -1.9995, -1.9873, -1.2603,
        -1.0427, -0.2675, -1.5361,  0.1526, -0.8788, -0.5716, -1.6364,  0.5365])

In [19]:
period_length_confidence, period_length = torch.max(torch.softmax(raw_period_length, dim=-1), dim=-1)
  

In [25]:
period_length < 2

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True])

In [48]:
rep_frames = plots.plot_repetitions(raw_frames[:len(best_period_count)], best_period_count.tolist(), best_periodicity_score.tolist() )
video = cv2.VideoWriter(os.path.join('./videos', 'repetitions.mp4'), cv2.VideoWriter_fourcc(*'mp4v'), fps, rep_frames[0].shape[:2][::-1])
for frame in rep_frames:
        video.write(frame)
video.release()